In [1]:
import requests
import pandas as pd
#import fsspec
from datetime import datetime, timedelta
from io import StringIO
import time
import numpy as np
import time
from ast import literal_eval


In [2]:
import json

In [3]:
leaguesDf = pd.read_csv('./leagues_parsed.csv')
leaguesDf = leaguesDf.drop(leaguesDf.columns[0], axis=1)

In [4]:
def matches_to_list(matches):
    matches = matches[2:-2]
    li = list(matches.split("\', \'"))
    return li

In [5]:
leaguesDf['matches'] = leaguesDf['matches'].apply(matches_to_list)

In [8]:
#WHEN APPENDING TO ALREADY EXISTING DF
matchesDf = pd.read_csv('46000_samples.csv')
matchesDf = matchesDf.drop(matchesDf.columns[0], axis=1)

In [74]:
#WHEN FIRST CREATING DF
matchesDf = pd.DataFrame()
matchesDf['match_id'] = 0
matchesDf['start_time'] = 0
matchesDf['league_id'] = 0
matchesDf['radiant_win'] = False
matchesDf['duration'] = 0
matchesDf['player_0'] = 0
matchesDf['player_1'] = 0
matchesDf['player_2'] = 0
matchesDf['player_3'] = 0
matchesDf['player_4'] = 0
matchesDf['player_5'] = 0
matchesDf['player_6'] = 0
matchesDf['player_7'] = 0
matchesDf['player_8'] = 0
matchesDf['player_9'] = 0

In [75]:
broken_games = []

In [82]:
def get_last_ids(matches):
    if matches.shape[0] == 0:
        return 0, 0
    else:
        last_league = matches.loc[matches.shape[0] - 1]['league_id']
        fl = True
        i = matches.shape[0] - 2
        j = 0
        while fl:
            if matches.loc[i]['league_id'] != last_league:
                fl = False
            i -= 1
            j += 1
        #print('last league id:', last_league, 'index in leaguesDf:', leaguesDf.loc[leaguesDf['league_id'] == last_league].index[0],  'games parsed:', j)
        return leaguesDf.loc[leaguesDf['league_id'] == last_league].index[0], j
        

In [94]:
matchesDf.shape[0]

128

In [43]:
get_last_ids(matchesDf)

last league id: 4874 index in leaguesDf: 530 games parsed: 47


In [7]:
matches = leaguesDf.loc[0]['matches']
payload = {'key': '4D52001F08216AA0FCD419BD3458DC28', 'match_id': matches[0]}
request_url = 'https://api.steampowered.com/IDOTA2Match_570/GetMatchDetails/V001'
r = requests.get(request_url, params=payload)
matchDf = pd.read_json(r.text)

In [8]:
matchDf

,result
barracks_status_dire,63
barracks_status_radiant,62
cluster,136
dire_captain,89137399
dire_logo,1740069961068193325
dire_name,Virtus.pro
dire_score,0
dire_team_complete,1
dire_team_id,1883502
duration,1816


In [89]:
#Add single league matches to matchesDf
start = time.time()
matches = leaguesDf.loc[get_last_ids(matchesDf)[0]]['matches']
print(str(len(matches)) + " matches in " + str(leaguesDf.loc[get_last_ids(matchesDf)[0]]['league_name']) + "\n\n\n")
for i in range(get_last_ids(matchesDf)[1], len(matches)):
    payload = {'key': '4D52001F08216AA0FCD419BD3458DC28', 'match_id': matches[i]}
    request_url = 'https://api.steampowered.com/IDOTA2Match_570/GetMatchDetails/V001'
    r = requests.get(request_url, params=payload)
    matchDf = pd.read_json(r.text)
    players = []
    all_players_exist = True
    for player in range(0, 10):
        if "account_id" in matchDf.loc['players']['result'][player]:
            playersDict = {
                "account_id": matchDf.loc['players']['result'][player]['account_id'],
                "player_slot": matchDf.loc['players']['result'][player]['player_slot'],
                "net_worth": matchDf.loc['players']['result'][player]['net_worth'],
                "kills": matchDf.loc['players']['result'][player]['kills'],
                "deaths": matchDf.loc['players']['result'][player]['deaths'],
                "assists": matchDf.loc['players']['result'][player]['assists'],
                "last_hits": matchDf.loc['players']['result'][player]['last_hits'],
                "denies": matchDf.loc['players']['result'][player]['denies']
            }
            players.append(playersDict)
        else:
            all_players_exist = False
    if all_players_exist:
        match = {'match_id': matches[i], 'start_time': matchDf.loc['start_time']['result'], 'league_id': matchDf.loc['leagueid']['result'], 'radiant_win': matchDf.loc['radiant_win']['result'], 'duration': matchDf.loc['duration']['result'], 'player_0': players[0], 'player_1': players[1], 'player_2': players[2], 'player_3': players[3], 'player_4': players[4], 'player_5': players[5], 'player_6': players[6], 'player_7': players[7], 'player_8': players[8], 'player_9': players[9]}
        matchesDf = matchesDf.append(match, ignore_index=True)
    else:
        print("ERRIR!!!!!!!!!!!!!!!!!!!!!!!!!")
        broken_games.append(matches[i])
    if i % 10 == 0:
        print("Parsed", i, 'matches.')
    end = time.time()
print(end - start, "seconds passed")


KeyError: -1

In [88]:
leaguesDf

,matches,league_id,league_ticket,league_banner,league_tier,league_name,games_num
0,"[909614535, 909523648, 909155180, 909024942, 9...",1761,NaN,NaN,professional,Excellent Moscow Cup Season 2,1408
1,"[1045688766, 1045586409, 1045442485, 104534795...",2061,NaN,NaN,professional,ESL Think Fast Razer League + HUD Skin,852
2,"[1337377384, 1337120175, 1336298304, 133717262...",2511,NaN,NaN,professional,Major Allstars tournament,1056
3,"[1730814626, 1725655035, 1728225922, 172566343...",3221,NaN,NaN,professional,On Art Challenge Series #2,636
4,"[4145826614, 4145986440, 4145705949, 414614096...",3455,NaN,NaN,professional,Elpadrinoth Latin Tournaments Season 2,264
...,...,...,...,...,...,...,...
991,"[32403902, 31939673, 30064720, 30242039, 29845...",4,econ/leagues/subscriptions_thedefense,econ/leagues/the_defense,professional,The Defense,1110
992,"[185954339, 179800433, 179751616, 176792062]",37,econ/leagues/subscriptions_atod3,econ/leagues/subscriptions_atod3_ingame,professional,AtoD 3,44
993,"[19934308, 19926255, 19910918, 19917771, 19896...",2,econ/tools/gift_multiple,econ/leagues/subscription_logo_the_wildcards,premium,Wild Cards West,350
994,"[957784984, 958652511, 958531303, 958446330, 9...",1245,NaN,NaN,professional,SuL,517


In [85]:
broken_games

[]

In [86]:
leaguesDf

,matches,league_id,league_ticket,league_banner,league_tier,league_name,games_num
0,"[909614535, 909523648, 909155180, 909024942, 9...",1761,NaN,NaN,professional,Excellent Moscow Cup Season 2,1408
1,"[1045688766, 1045586409, 1045442485, 104534795...",2061,NaN,NaN,professional,ESL Think Fast Razer League + HUD Skin,852
2,"[1337377384, 1337120175, 1336298304, 133717262...",2511,NaN,NaN,professional,Major Allstars tournament,1056
3,"[1730814626, 1725655035, 1728225922, 172566343...",3221,NaN,NaN,professional,On Art Challenge Series #2,636
4,"[4145826614, 4145986440, 4145705949, 414614096...",3455,NaN,NaN,professional,Elpadrinoth Latin Tournaments Season 2,264
...,...,...,...,...,...,...,...
991,"[32403902, 31939673, 30064720, 30242039, 29845...",4,econ/leagues/subscriptions_thedefense,econ/leagues/the_defense,professional,The Defense,1110
992,"[185954339, 179800433, 179751616, 176792062]",37,econ/leagues/subscriptions_atod3,econ/leagues/subscriptions_atod3_ingame,professional,AtoD 3,44
993,"[19934308, 19926255, 19910918, 19917771, 19896...",2,econ/tools/gift_multiple,econ/leagues/subscription_logo_the_wildcards,premium,Wild Cards West,350
994,"[957784984, 958652511, 958531303, 958446330, 9...",1245,NaN,NaN,professional,SuL,517


In [63]:
start = time.time()
print("ERROR")
for index, row in leaguesDf.iloc[get_last_ids(matchesDf)[0] + 1:].iterrows():
    matches = row['matches']
    print(len(matches), "matches in", row['league_name'], "league. League num = ", index, "\n\n\n")
    for i in range(0, len(matches)):
        payload = {'key': '4D52001F08216AA0FCD419BD3458DC28', 'match_id': matches[i]}
        request_url = 'https://api.steampowered.com/IDOTA2Match_570/GetMatchDetails/V001'
        r = requests.get(request_url, params=payload)
        try:
            matchDf = pd.read_json(r.text)
        except ValueError:
            print("\n\n\nERROR!!!!!!!!!!!!!!!!!\nLeague_id == " + str(index) + ". Match num:" + str(i) + "\n\n\n")
            break
                
        players = []
        all_players_exist = True
        for player in range(0, 10):
            if "account_id" in matchDf.loc['players']['result'][player]:
                playersDict = {
                    "account_id": matchDf.loc['players']['result'][player]['account_id'],
                    "player_slot": matchDf.loc['players']['result'][player]['player_slot'],
                    "net_worth": matchDf.loc['players']['result'][player]['net_worth'],
                    "kills": matchDf.loc['players']['result'][player]['kills'],
                    "deaths": matchDf.loc['players']['result'][player]['deaths'],
                    "assists": matchDf.loc['players']['result'][player]['assists'],
                    "last_hits": matchDf.loc['players']['result'][player]['last_hits'],
                    "denies": matchDf.loc['players']['result'][player]['denies']
                }
                players.append(playersDict)
            else:
                all_players_exist = False
        if all_players_exist:
            match = {'match_id': matches[i], 'start_time': matchDf.loc['start_time']['result'], 'league_id': matchDf.loc['leagueid']['result'], 'radiant_win': matchDf.loc['radiant_win']['result'], 'duration': matchDf.loc['duration']['result'], 'player_0': players[0], 'player_1': players[1], 'player_2': players[2], 'player_3': players[3], 'player_4': players[4], 'player_5': players[5], 'player_6': players[6], 'player_7': players[7], 'player_8': players[8], 'player_9': players[9]}
            matchesDf = matchesDf.append(match, ignore_index=True)
        else:
            broken_games.append(matches[i])
        if i % 10 == 0:
            print("Parsed", i, 'matches.')
    end = time.time()
    print(index, end - start, "seconds passed")
    if index % 100 == 0:
        matchesDf.to_csv('./' + str(matchesDf.shape[0]) + 'samples.csv')
        

ERROR


In [68]:
matchesDf.to_csv('full_first_time.csv')

In [33]:
matchesDf.to_csv('29000_samples.csv')

In [30]:
matchesDf = matchesDf[:23722]

In [32]:
matchesDf

,match_id,start_time,league_id,radiant_win,duration,player_0,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9
0,909614535,1411238761,1761,False,1816,"{'account_id': 49317728, 'player_slot': 0, 'ne...","{'account_id': 12231202, 'player_slot': 1, 'ne...","{'account_id': 38628747, 'player_slot': 2, 'ne...","{'account_id': 50828662, 'player_slot': 3, 'ne...","{'account_id': 44111721, 'player_slot': 4, 'ne...","{'account_id': 89137399, 'player_slot': 128, '...","{'account_id': 26316691, 'player_slot': 129, '...","{'account_id': 86750262, 'player_slot': 130, '...","{'account_id': 94338967, 'player_slot': 131, '...","{'account_id': 106809101, 'player_slot': 132, ..."
1,909523648,1411236053,1761,True,1351,"{'account_id': 89137399, 'player_slot': 0, 'ne...","{'account_id': 26316691, 'player_slot': 1, 'ne...","{'account_id': 86750262, 'player_slot': 2, 'ne...","{'account_id': 94338967, 'player_slot': 3, 'ne...","{'account_id': 106809101, 'player_slot': 4, 'n...","{'account_id': 49317728, 'player_slot': 128, '...","{'account_id': 12231202, 'player_slot': 129, '...","{'account_id': 38628747, 'player_slot': 130, '...","{'account_id': 50828662, 'player_slot': 131, '...","{'account_id': 44111721, 'player_slot': 132, '..."
2,909155180,1411226737,1761,True,1888,"{'account_id': 94049589, 'player_slot': 0, 'ne...","{'account_id': 89625472, 'player_slot': 1, 'ne...","{'account_id': 70388657, 'player_slot': 2, 'ne...","{'account_id': 11550182, 'player_slot': 3, 'ne...","{'account_id': 86723143, 'player_slot': 4, 'ne...","{'account_id': 88271237, 'player_slot': 128, '...","{'account_id': 29337472, 'player_slot': 129, '...","{'account_id': 40013910, 'player_slot': 130, '...","{'account_id': 16769223, 'player_slot': 131, '...","{'account_id': 99845008, 'player_slot': 132, '..."
3,909024942,1411223777,1761,False,1876,"{'account_id': 88271237, 'player_slot': 0, 'ne...","{'account_id': 29337472, 'player_slot': 1, 'ne...","{'account_id': 40013910, 'player_slot': 2, 'ne...","{'account_id': 16769223, 'player_slot': 3, 'ne...","{'account_id': 99845008, 'player_slot': 4, 'ne...","{'account_id': 94049589, 'player_slot': 128, '...","{'account_id': 89625472, 'player_slot': 129, '...","{'account_id': 70388657, 'player_slot': 130, '...","{'account_id': 11550182, 'player_slot': 131, '...","{'account_id': 86723143, 'player_slot': 132, '..."
4,908871129,1411220388,1761,True,2007,"{'account_id': 21265941, 'player_slot': 0, 'ne...","{'account_id': 72312627, 'player_slot': 1, 'ne...","{'account_id': 24009418, 'player_slot': 2, 'ne...","{'account_id': 8712306, 'player_slot': 3, 'net...","{'account_id': 85375207, 'player_slot': 4, 'ne...","{'account_id': 96169991, 'player_slot': 128, '...","{'account_id': 146772593, 'player_slot': 129, ...","{'account_id': 97590558, 'player_slot': 130, '...","{'account_id': 26682464, 'player_slot': 131, '...","{'account_id': 53151216, 'player_slot': 132, '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28725,4784706293,1558847050,10684,False,1890,"{'account_id': 97048475, 'player_slot': 0, 'ne...","{'account_id': 102070845, 'player_slot': 1, 'n...","{'account_id': 197098235, 'player_slot': 2, 'n...","{'account_id': 112297739, 'player_slot': 3, 'n...","{'account_id': 121532383, 'player_slot': 4, 'n...","{'account_id': 89603649, 'player_slot': 128, '...","{'account_id': 66598119, 'player_slot': 129, '...","{'account_id': 76904792, 'player_slot': 130, '...","{'account_id': 89654154, 'player_slot': 131, '...","{'account_id': 163392533, 'player_slot': 132, ..."
28726,4785255999,1558859409,10684,True,1322,"{'account_id': 89603649, 'player_slot': 0, 'ne...","{'account_id': 66598119, 'player_slot': 1, 'ne...","{'account_id': 76904792, 'player_slot': 2, 'ne...","{'account_id': 89654154, 'player_slot': 3, 'ne...","{'account_id': 163392533, 'player_slot': 4, 'n...","{'account_id': 142536387, 'player_slot': 128, ...","{'account_id': 108030695, 'player_slot': 129, ...","{'account_id': 84234671, 'pl

In [20]:
leaguesDf.loc[63]

matches          [3010688520, 3010640333, 2987661229, 298756417...
league_id                                                     4644
league_ticket                                                  NaN
league_banner                                                  NaN
league_tier                                           professional
league_name                           CZ-SK Dota 2 League Season 8
games_num                                                      552
Name: 63, dtype: object

In [23]:
matchesDf

,match_id,start_time,league_id,radiant_win,duration,player_0,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9
0,909614535,1411238761,1761,False,1816,"{'account_id': 49317728, 'player_slot': 0, 'ne...","{'account_id': 12231202, 'player_slot': 1, 'ne...","{'account_id': 38628747, 'player_slot': 2, 'ne...","{'account_id': 50828662, 'player_slot': 3, 'ne...","{'account_id': 44111721, 'player_slot': 4, 'ne...","{'account_id': 89137399, 'player_slot': 128, '...","{'account_id': 26316691, 'player_slot': 129, '...","{'account_id': 86750262, 'player_slot': 130, '...","{'account_id': 94338967, 'player_slot': 131, '...","{'account_id': 106809101, 'player_slot': 132, ..."
1,909523648,1411236053,1761,True,1351,"{'account_id': 89137399, 'player_slot': 0, 'ne...","{'account_id': 26316691, 'player_slot': 1, 'ne...","{'account_id': 86750262, 'player_slot': 2, 'ne...","{'account_id': 94338967, 'player_slot': 3, 'ne...","{'account_id': 106809101, 'player_slot': 4, 'n...","{'account_id': 49317728, 'player_slot': 128, '...","{'account_id': 12231202, 'player_slot': 129, '...","{'account_id': 38628747, 'player_slot': 130, '...","{'account_id': 50828662, 'player_slot': 131, '...","{'account_id': 44111721, 'player_slot': 132, '..."
2,909155180,1411226737,1761,True,1888,"{'account_id': 94049589, 'player_slot': 0, 'ne...","{'account_id': 89625472, 'player_slot': 1, 'ne...","{'account_id': 70388657, 'player_slot': 2, 'ne...","{'account_id': 11550182, 'player_slot': 3, 'ne...","{'account_id': 86723143, 'player_slot': 4, 'ne...","{'account_id': 88271237, 'player_slot': 128, '...","{'account_id': 29337472, 'player_slot': 129, '...","{'account_id': 40013910, 'player_slot': 130, '...","{'account_id': 16769223, 'player_slot': 131, '...","{'account_id': 99845008, 'player_slot': 132, '..."
3,909024942,1411223777,1761,False,1876,"{'account_id': 88271237, 'player_slot': 0, 'ne...","{'account_id': 29337472, 'player_slot': 1, 'ne...","{'account_id': 40013910, 'player_slot': 2, 'ne...","{'account_id': 16769223, 'player_slot': 3, 'ne...","{'account_id': 99845008, 'player_slot': 4, 'ne...","{'account_id': 94049589, 'player_slot': 128, '...","{'account_id': 89625472, 'player_slot': 129, '...","{'account_id': 70388657, 'player_slot': 130, '...","{'account_id': 11550182, 'player_slot': 131, '...","{'account_id': 86723143, 'player_slot': 132, '..."
4,908871129,1411220388,1761,True,2007,"{'account_id': 21265941, 'player_slot': 0, 'ne...","{'account_id': 72312627, 'player_slot': 1, 'ne...","{'account_id': 24009418, 'player_slot': 2, 'ne...","{'account_id': 8712306, 'player_slot': 3, 'net...","{'account_id': 85375207, 'player_slot': 4, 'ne...","{'account_id': 96169991, 'player_slot': 128, '...","{'account_id': 146772593, 'player_slot': 129, ...","{'account_id': 97590558, 'player_slot': 130, '...","{'account_id': 26682464, 'player_slot': 131, '...","{'account_id': 53151216, 'player_slot': 132, '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8533,2418282767,1465265653,4618,True,1865,"{'account_id': 177885192, 'player_slot': 0, 'n...","{'account_id': 99803639, 'player_slot': 1, 'ne...","{'account_id': 87419473, 'player_slot': 2, 'ne...","{'account_id': 36327727, 'player_slot': 3, 'ne...","{'account_id': 102260924, 'player_slot': 4, 'n...","{'account_id': 152995826, 'player_slot': 128, ...","{'account_id': 124099683, 'player_slot': 129, ...","{'account_id': 67902269, 'player_slot': 130, '...","{'account_id': 112891883, 'player_slot': 131, ...","{'account_id': 35242434, 'player_slot': 132, '..."
8534,3297548442,1499392201,4618,True,2419,"{'account_id': 87040694, 'player_slot': 0, 'ne...","{'account_id': 135553543, 'player_slot': 1, 'n...","{'account_id': 412758827, 'player_slot': 2, 'n...","{'account_id': 87419473, 'player_slot': 3, 'ne...","{'account_id': 36327727, 'player_slot': 4, 'ne...","{'account_id': 67902269, 'player_slot': 128, '...","{'account_id': 371313183, 'player_slot': 129, ...","{'account_id': 124099683, 'player

In [256]:
matchesDf.to_csv('8500_matches_parsed.csv')
broken_games

In [ ]:
from playsound import playsound

playsound('beard_trimmer_running.mp3')

In [245]:
import multiprocessing
from playsound import playsound

p = multiprocessing.Process(target=playsound, args=('beard_trimmer_running.mp3',))
p.start()
input("press ENTER to stop playback")
p.terminate()

Process Process-4:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
TypeError: _playsoundOSX() takes from 1 to 2 positional arguments but 18 were given


press ENTER to stop playback


In [253]:
matchesDf = matchesDf[:8111]

In [252]:
matchesDf

,match_id,start_time,league_id,radiant_win,duration,player_0,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9
0,909614535,1411238761,1761,False,1816,"{'account_id': 49317728, 'player_slot': 0, 'ne...","{'account_id': 12231202, 'player_slot': 1, 'ne...","{'account_id': 38628747, 'player_slot': 2, 'ne...","{'account_id': 50828662, 'player_slot': 3, 'ne...","{'account_id': 44111721, 'player_slot': 4, 'ne...","{'account_id': 89137399, 'player_slot': 128, '...","{'account_id': 26316691, 'player_slot': 129, '...","{'account_id': 86750262, 'player_slot': 130, '...","{'account_id': 94338967, 'player_slot': 131, '...","{'account_id': 106809101, 'player_slot': 132, ..."
1,909523648,1411236053,1761,True,1351,"{'account_id': 89137399, 'player_slot': 0, 'ne...","{'account_id': 26316691, 'player_slot': 1, 'ne...","{'account_id': 86750262, 'player_slot': 2, 'ne...","{'account_id': 94338967, 'player_slot': 3, 'ne...","{'account_id': 106809101, 'player_slot': 4, 'n...","{'account_id': 49317728, 'player_slot': 128, '...","{'account_id': 12231202, 'player_slot': 129, '...","{'account_id': 38628747, 'player_slot': 130, '...","{'account_id': 50828662, 'player_slot': 131, '...","{'account_id': 44111721, 'player_slot': 132, '..."
2,909155180,1411226737,1761,True,1888,"{'account_id': 94049589, 'player_slot': 0, 'ne...","{'account_id': 89625472, 'player_slot': 1, 'ne...","{'account_id': 70388657, 'player_slot': 2, 'ne...","{'account_id': 11550182, 'player_slot': 3, 'ne...","{'account_id': 86723143, 'player_slot': 4, 'ne...","{'account_id': 88271237, 'player_slot': 128, '...","{'account_id': 29337472, 'player_slot': 129, '...","{'account_id': 40013910, 'player_slot': 130, '...","{'account_id': 16769223, 'player_slot': 131, '...","{'account_id': 99845008, 'player_slot': 132, '..."
3,909024942,1411223777,1761,False,1876,"{'account_id': 88271237, 'player_slot': 0, 'ne...","{'account_id': 29337472, 'player_slot': 1, 'ne...","{'account_id': 40013910, 'player_slot': 2, 'ne...","{'account_id': 16769223, 'player_slot': 3, 'ne...","{'account_id': 99845008, 'player_slot': 4, 'ne...","{'account_id': 94049589, 'player_slot': 128, '...","{'account_id': 89625472, 'player_slot': 129, '...","{'account_id': 70388657, 'player_slot': 130, '...","{'account_id': 11550182, 'player_slot': 131, '...","{'account_id': 86723143, 'player_slot': 132, '..."
4,908871129,1411220388,1761,True,2007,"{'account_id': 21265941, 'player_slot': 0, 'ne...","{'account_id': 72312627, 'player_slot': 1, 'ne...","{'account_id': 24009418, 'player_slot': 2, 'ne...","{'account_id': 8712306, 'player_slot': 3, 'net...","{'account_id': 85375207, 'player_slot': 4, 'ne...","{'account_id': 96169991, 'player_slot': 128, '...","{'account_id': 146772593, 'player_slot': 129, ...","{'account_id': 97590558, 'player_slot': 130, '...","{'account_id': 26682464, 'player_slot': 131, '...","{'account_id': 53151216, 'player_slot': 132, '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8108,2126057981,1454530483,3454,False,1914,"{'account_id': 117015167, 'player_slot': 0, 'n...","{'account_id': 86664345, 'player_slot': 1, 'ne...","{'account_id': 169025618, 'player_slot': 2, 'n...","{'account_id': 86896635, 'player_slot': 3, 'ne...","{'account_id': 92968238, 'player_slot': 4, 'ne...","{'account_id': 99539136, 'player_slot': 128, '...","{'account_id': 95439451, 'player_slot': 129, '...","{'account_id': 228916210, 'player_slot': 130, ...","{'account_id': 87197791, 'player_slot': 131, '...","{'account_id': 86740536, 'player_slot': 132, '..."
8109,2125638814,1454515490,3454,False,1844,"{'account_id': 86725175, 'player_slot': 0, 'ne...","{'account_id': 89625472, 'player_slot': 1, 'ne...","{'account_id': 93552791, 'player_slot': 2, 'ne...","{'account_id': 123051238, 'player_slot': 3, 'n...","{'account_id': 86723143, 'player_slot': 4, 'ne...","{'account_id': 134556694, 'player_slot': 128, ...","{'account_id': 91460772, 'player_slot': 129, '...","{'account_id': 106573901, 'play

In [193]:
import json
json.loads(r.text)

{'result': {'players': [{'account_id': 154846792,
    'player_slot': 0,
    'hero_id': 50,
    'item_0': 254,
    'item_1': 187,
    'item_2': 36,
    'item_3': 46,
    'item_4': 180,
    'item_5': 42,
    'backpack_0': 0,
    'backpack_1': 0,
    'backpack_2': 0,
    'item_neutral': 0,
    'kills': 1,
    'deaths': 9,
    'assists': 7,
    'leaver_status': 0,
    'last_hits': 43,
    'denies': 3,
    'gold_per_min': 192,
    'xp_per_min': 218,
    'level': 14,
    'net_worth': 5852,
    'aghanims_scepter': 0,
    'aghanims_shard': 0,
    'moonshard': 0},
   {'account_id': 123858122,
    'player_slot': 1,
    'hero_id': 17,
    'item_0': 63,
    'item_1': 98,
    'item_2': 121,
    'item_3': 46,
    'item_4': 123,
    'item_5': 119,
    'backpack_0': 0,
    'backpack_1': 0,
    'backpack_2': 0,
    'item_neutral': 0,
    'kills': 13,
    'deaths': 6,
    'assists': 13,
    'leaver_status': 0,
    'last_hits': 287,
    'denies': 6,
    'gold_per_min': 513,
    'xp_per_min': 678,
    'le

In [189]:
r.text

'{\n"result":{\n"players":[\n{\n"account_id":154846792,\n"player_slot":0,\n"hero_id":50,\n"item_0":254,\n"item_1":187,\n"item_2":36,\n"item_3":46,\n"item_4":180,\n"item_5":42,\n"backpack_0":0,\n"backpack_1":0,\n"backpack_2":0,\n"item_neutral":0,\n"kills":1,\n"deaths":9,\n"assists":7,\n"leaver_status":0,\n"last_hits":43,\n"denies":3,\n"gold_per_min":192,\n"xp_per_min":218,\n"level":14,\n"net_worth":5852,\n"aghanims_scepter":0,\n"aghanims_shard":0,\n"moonshard":0\n},\n{\n"account_id":123858122,\n"player_slot":1,\n"hero_id":17,\n"item_0":63,\n"item_1":98,\n"item_2":121,\n"item_3":46,\n"item_4":123,\n"item_5":119,\n"backpack_0":0,\n"backpack_1":0,\n"backpack_2":0,\n"item_neutral":0,\n"kills":13,\n"deaths":6,\n"assists":13,\n"leaver_status":0,\n"last_hits":287,\n"denies":6,\n"gold_per_min":513,\n"xp_per_min":678,\n"level":25,\n"net_worth":21489,\n"aghanims_scepter":0,\n"aghanims_shard":0,\n"moonshard":0\n},\n{\n"account_id":126172693,\n"player_slot":2,\n"hero_id":39,\n"item_0":158,\n"item_1

In [103]:
testNum

0

In [121]:
payload = {'key': '4D52001F08216AA0FCD419BD3458DC28', 'match_id': leaguesDf.loc[0]['matches'][1]}
request_url = 'https://api.steampowered.com/IDOTA2Match_570/GetMatchDetails/V001'
r = requests.get(request_url, params=payload)
pd.read_json(r.text)

,result
barracks_status_dire,63
barracks_status_radiant,63
cluster,133
dire_captain,49317728
dire_logo,802116058284669897
dire_name,compLexity Gaming
dire_score,0
dire_team_complete,1
dire_team_id,3
duration,1351


In [122]:
r.text.find('start_time')

4215

In [39]:
test = pd.read_json(r.text)

In [40]:
test

,result
barracks_status_dire,63
barracks_status_radiant,63
cluster,152
dire_captain,248388120
dire_logo,947336408508413358
dire_name,Team Orca
dire_score,23
dire_team_complete,1
dire_team_id,5994455
duration,1864


In [ ]:
thisdict = {
  "account_id": "Ford",
  "model": "Mustang",
  "year": 1964
}

In [43]:
test.loc['players']['result'][0]

{'account_id': 92620439,
 'player_slot': 0,
 'hero_id': 102,
 'item_0': 187,
 'item_1': 188,
 'item_2': 11,
 'item_3': 214,
 'item_4': 218,
 'item_5': 36,
 'backpack_0': 0,
 'backpack_1': 0,
 'backpack_2': 0,
 'item_neutral': 0,
 'kills': 0,
 'deaths': 6,
 'assists': 20,
 'leaver_status': 0,
 'last_hits': 10,
 'denies': 3,
 'gold_per_min': 217,
 'xp_per_min': 436,
 'level': 17,
 'net_worth': 3755,
 'aghanims_scepter': 0,
 'aghanims_shard': 0,
 'moonshard': 0}

In [37]:
matchesDf.loc[0]['player_0']

"{'account_id': 49317728, 'player_slot': 0, 'net_worth': 9885, 'kills': 1, 'deaths': 6, 'assists': 7, 'last_hits': 151, 'denies': 5}"